In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import  preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold

In [3]:
#try xgboost
#fucntion from SRK
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=10000,early_stopping = 100):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y,feature_names=feature_names)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y,feature_names=feature_names)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=early_stopping)
    else:
        xgtest = xgb.DMatrix(test_X,feature_names=feature_names)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [5]:
#lodaing data
data_path = "../../kaggleData/2sigma/"
train_file = data_path + "train.json"
test_file = data_path + "test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)


(49352, 15)
(74659, 14)


In [6]:
#basic numerical features
features_to_use  = ["bathrooms", "bedrooms", "latitude", "longitude", "price"]

In [7]:
#some transfromed features
# count of photos #
train_df["num_photos"] = train_df["photos"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

# count of "features" #
train_df["num_features"] = train_df["features"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)

# count of words present in description column #
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

# convert the created column to datetime object so as to extract more features 
train_df["created"] = pd.to_datetime(train_df["created"])
test_df["created"] = pd.to_datetime(test_df["created"])

# Let us extract some features like year, month, day, hour from date columns #
train_df["created_year"] = train_df["created"].dt.year
test_df["created_year"] = test_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
test_df["created_month"] = test_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
test_df["created_day"] = test_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour
test_df["created_hour"] = test_df["created"].dt.hour

# adding all these new features to use list # "listing_id",
features_to_use.extend(["num_photos", "num_features", "num_description_words","created_year","listing_id", "created_month", "created_day", "created_hour"])

In [8]:
#dealing feature with categorical features 
"""
display_address 8826    
building_id        7585   =》many zeros in this feature
manager_id   3481
street_address 15358 =》will be 3800 if no numbers in it 
"""
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train_df[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            features_to_use.append(f)

In [9]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_X = train_df[features_to_use]
test_X = test_df[features_to_use]

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_X),5,shuffle=True,random_state = 42)

In [10]:
#running and getting the cv from xgboost
cv_scores = []
#K-FOLD already defined.If not ,use
#KF=KFold(len(train_X),5,shuffle=True,random_state = 42)
for dev_index, val_index in KF:
        dev_X, val_X = train_X.iloc[dev_index,:].as_matrix(), train_X.iloc[val_index,:].as_matrix()
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        preds, model = runXGB(dev_X, dev_y, val_X, val_y,feature_names=list(list(train_X.columns)))
        cv_scores.append(log_loss(val_y, preds))
        print(cv_scores)
        break

[0]	train-mlogloss:1.04088	test-mlogloss:1.04249
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 100 rounds.
[1]	train-mlogloss:0.990776	test-mlogloss:0.993616
[2]	train-mlogloss:0.950587	test-mlogloss:0.95463
[3]	train-mlogloss:0.913564	test-mlogloss:0.918785
[4]	train-mlogloss:0.88242	test-mlogloss:0.888547
[5]	train-mlogloss:0.852429	test-mlogloss:0.859641
[6]	train-mlogloss:0.827476	test-mlogloss:0.835621
[7]	train-mlogloss:0.805708	test-mlogloss:0.814649
[8]	train-mlogloss:0.787008	test-mlogloss:0.796693
[9]	train-mlogloss:0.76903	test-mlogloss:0.779473
[10]	train-mlogloss:0.754415	test-mlogloss:0.765748
[11]	train-mlogloss:0.740433	test-mlogloss:0.752391
[12]	train-mlogloss:0.72925	test-mlogloss:0.741788
[13]	train-mlogloss:0.718262	test-mlogloss:0.73148
[14]	train-mlogloss:0.70762	test-mlogloss:0.721684
[15]	train-mlogloss:0.697077	test-mlogloss:0.71185
[16]	train-mlogloss:0.687149	test-m

In [9]:
#ananlysis by the feature importance by weight
weight = model.get_score()
total = sum(weight.values())
for key in weight:
    weight[key] = weight[key]*1.0/total
weight

{'bathrooms': 0.010285346346753424,
 'bedrooms': 0.030466314219872576,
 'building_id': 0.06857462383082553,
 'created_day': 0.046309475396502646,
 'created_hour': 0.04346278975193168,
 'created_month': 0.006015317879896977,
 'display_address': 0.0806899823776603,
 'latitude': 0.09202589128371967,
 'listing_id': 0.09887149247661652,
 'longitude': 0.07911413853870138,
 'manager_id': 0.09904093805069812,
 'num_description_words': 0.0829605530703538,
 'num_features': 0.04493696624644164,
 'num_photos': 0.04109055171478921,
 'price': 0.0970753693913515,
 'street_address': 0.07908024942388504}

In [ ]:
#ananlysis by the feature importance by gain
gain = model.get_score(importance_type='gain')
total = sum(gain.values())
#for key in gain:
#    gain[key] = gain[key]*1.0/total
gain

In [11]:
#ananlysis by the feature importance by coverage
cover = model.get_score(importance_type='cover')
total = sum(cover.values())
for key in cover:
    cover[key] = cover[key]*1.0/total
cover

{'bathrooms': 0.15003324661763429,
 'bedrooms': 0.11847222747849985,
 'building_id': 0.05966144646752775,
 'created_day': 0.027908091350767217,
 'created_hour': 0.04913703475375256,
 'created_month': 0.015463921187964249,
 'display_address': 0.051917534421511584,
 'latitude': 0.062329192852910546,
 'listing_id': 0.05823796559748455,
 'longitude': 0.05796867229011468,
 'manager_id': 0.0658834209429622,
 'num_description_words': 0.04385875263322271,
 'num_features': 0.05493240649113651,
 'num_photos': 0.053803480057786596,
 'price': 0.07955324745771991,
 'street_address': 0.050839359399004566}

In [8]:
#output the outcome - using xgboost
train_X_m = train_X.as_matrix()
test_X_m = test_X.as_matrix()

preds, model = runXGB(train_X_m, train_y, test_X_m, num_rounds=400)
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("xgb_beta1_benchmark.csv", index=False)